TODO: -Duplicatebereinigung
-Date
-CSV-Append realisieren

In [ ]:
df_final = pd.DataFrame()
soup_size_price = pd.DataFrame()
soup_address = pd.DataFrame()
temp_price_size= pd.DataFrame()
temp_address= pd.DataFrame()
final_frame= pd.DataFrame()
final_df = pd.DataFrame()
summary_df = pd.DataFrame()
sleep_sec=12
#30
for i in range(20,22):
   
   print('Round 1:'+str(i))
   #https://www.wg-gesucht.de/wg-zimmer-und-1-zimmer-wohnungen-und-wohnungen-in-Leipzig.77.0+1+2.1.0.html?offer_filter=1&city_id=77&sort_column=6&sort_order=0&noDeact=1&categories%5B%5D=0&categories%5B%5D=1&categories%5B%5D=2&rent_types%5B%5D=2&rent_types%5B%5D=2
   Url = 'https://www.wg-gesucht.de/wg-zimmer-und-1-zimmer-wohnungen-und-wohnungen-in-Leipzig.77.0+1+2.1.'+str(i)+'.html'
   print(Url)


   soup = get_soup(Url)
   #print(soup)

   #returns a df with the link of each flat
   link=get_subpages_from_soup(soup)
   #returns all information of the page lists
   front_page=get_front_page_data(soup)
   #concats list url with fronpage data
   front_frame= concat_link_address(link,front_page)
   #return all data from each subpage
   back_frame = get_sub_page_alpha(soup)
   #connects the front and backframe via the key url and link
   final_df= back_frame.merge(front_frame, left_on='Url', right_on='link')

   summary_df = pd.concat([summary_df,final_df])
   #waiting time between pages
   time.sleep(randint(1*100, sleep_sec*100)/100)
   #final_df= pd.concat([front_frame,get_sub_page_alpha(soup)])

In [ ]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
from random import randint
from geopy.geocoders import Nominatim
import geopandas as gpd
import time
import re
import numpy as np
import geopandas as gpd
import traceback
import datetime
import sys

In [ ]:
display(summary_df)

In [ ]:
#Test stuff checking each variable
#----------------------------------
#display(soup_size_price)
#display(soup_address)
#display(temp_price_size)
#display(temp_address)
#display(final_frame)
#display(back_frame)
#display(final_df)
#len(summary_df)
display(summary_df)

   

In [ ]:
display(summary_df)

Safe Data in CSV

In [ ]:
#only relevant for the first time 
summary_df.to_csv('Summary_subpages19.csv',index=False)

In [ ]:
#Append Dataframe to existing CSV
summary_df.to_csv('Summary_subpages2.csv', mode='a',index=False, header=False)

print("DataFrame wurde erfolgreich an die CSV-Datei angehängt.")

In [ ]:
df_pickeled = pd.read_csv('Summary_subpages2.csv',delimiter=',')

In [ ]:
df_duplicate_free = pd.read_csv('Summary_subpages2.csv',delimiter=',')


In [ ]:
new=df_duplicate_free.drop_duplicates(subset=['Url'], keep='first', inplace=False, ignore_index=False)
new_dup_free=new.dropna(subset=['Street', 'Address','rent_cold','additional_cost'])
len(new)


In [ ]:
new_dup_free.to_csv('Summary_subpages_duplicate_free.csv',index=False)

In [ ]:
print(new['Street'].value_counts(dropna=False))

In [ ]:
len(df_pickeled)

In [ ]:

display(df_pickeled)


In [ ]:

df_homes = pd.read_csv("Summary_subpages2.csv")
df_homes1 = pd.read_csv("C:/Users/Oliver/Desktop/Unizeug/Master/2.Semester/Spatial/Summary_subpages.csv")
pd.concat([df_homes, df_homes1]).to_csv('homes_complete.csv', index=False)

ADD Map data
#Additional task cos nominatim service fails sometimes

In [ ]:
#Add long lat Data to geo_df
#print´s address if legit address available 
geo_df= get_geo_location(df_pickeled)

In [ ]:
geo_df= get_geo_location(new_dup_free)
#new_dup_free

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
display(geo_df)

Second CSV for safety reasons^^

In [ ]:
#only relevant for the first time  creates CSV File
#
geo_df.to_csv('Summary_subpages_w_location2.csv',index=False)

In [ ]:
geo_df_pickeled = pd.read_csv('Summary_subpages_w_location2.csv',delimiter=',')

In [ ]:
geo_df_pickeled

#################################################################################################
Functionsblock
#################################################################################################

In [ ]:
def get_geo_location(df_wg_ges):
    import geopy
    user_agent = 'user_me_{}'.format(randint(10000,99999))
    geolocator = Nominatim(user_agent=user_agent)
    sleep_sec=1
    
    #parsing problem troubleshooting, otherwise pyhton creates string with float value...
    df_wg_ges['location'] = 0.0
    df_wg_ges['latitude'] = 0.0
    df_wg_ges['longitude'] = 0.0

    for inx in range(len(df_wg_ges)):
        print(inx)
        if(len(str(df_wg_ges['Street'].iat[inx]))==0 and len(str(df_wg_ges['Address'].iat[inx])) ==0):
            continue
        temp=str(df_wg_ges['Street'].iat[inx] +' '+ df_wg_ges['Address'].iat[inx])
        time.sleep(randint(1*100, sleep_sec*100)/100)
        location = geolocator.geocode(temp)
        print(location)
        if(location==None):
            continue
        df_wg_ges['location'].iat[inx] = location
        df_wg_ges['latitude'].iat[inx] = location.latitude
        df_wg_ges['longitude'].iat[inx] = location.longitude
    
    return df_wg_ges

In [ ]:
def concat_link_address(link,temp_address):
     temp_address['link']=link
     return temp_address

In [ ]:
#create Soup from mainpage # I#m using headful browsing for avoiding bot-blocks

def get_soup(url):
    
    print('current URL: '+str(url))
    #url="https://www.wg-gesucht.de/1-zimmer-wohnungen-in-Leipzig.77.1.1.0.html"

    result = requests.get(url,headers = {'User-Agent':'Mozilla/5.0'})
    soup = BeautifulSoup(result.text, 'html.parser') ##lxml as parser
    return soup


In [ ]:
"""from requests_tor import RequestsTor
def get_soup(url):
    rt = RequestsTor(tor_ports=(9050,), tor_cport=9051)

    #url="https://www.wg-gesucht.de/1-zimmer-wohnungen-in-Leipzig.77.1.1.0.html"
    params = {
        "id": 12345,
        "status": 'passed'
        }
    headers = {
        "Origin": "https://www.wg-gesucht.de",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
        }
    result = rt.get(url, params=params, headers=headers)
    #print(r.text)
    soup = BeautifulSoup(result.text, 'html.parser')
    return soup
"""

In [ ]:
def get_sub_page_alpha(soup):
    
    flats = soup.find_all(
            class_="truncate_title noprint"
        )
    
    sleep_sec=1
    #place holder lists for every feature
    list_accessible = []
    list_rent_cold = []
    list_additional_cost = []
    list_distance = []
    list_gadgets = []
    list_female_roommates = []
    list_male_roommates = []
    list_divers_roommates = []
    list_room_description = []
    list_location_description = []
    list_flat_live = []
    list_additional = []
    list_smoker =[]
    list_kind_of_flat = []
    list_url = []
    list_free_from = []
    list_plz_city= []
    list_street = []

   
    for idx,flat in enumerate (flats):
            
            time.sleep(randint(1*100, sleep_sec*100)/100)
            #navigate to subpage and create subpage_Url
            flat_url = str('https://www.wg-gesucht.de')+str(flat.find('a')['href'])
            if 'asset_id' in flat_url:
                continue
            #flat_url = str('https://www.wg-gesucht.de/wg-zimmer-in-Leipzig-Volkmarsdorf.5200825.html')
            #create soup of subpage
            flat_soup = get_soup(flat_url)

            #invoke scraped text funktions
            #using BS4 funtions to shape the soup
            try:
                rent_cold_additional_cost = flat_soup.find(class_= 'col-sm-5').text.replace('\n', '').replace('   ', '')
            except:
                rent_cold_additional_cost = ''
                pass

            try:
                if('Barrierefrei'== flat_soup.find(class_= 'mdi mdi-wheelchair-accessibility mdi-36px noprint').parent.text.replace('\n', '').replace(' ', '') ):
                    accessible = 1
            except:
                accessible = 0
                pass
                
            try:
                distance = flat_soup.find(class_= 'mdi mdi-bus mdi-36px noprint').parent.text.replace('\n', '').replace('   ', '')
            except:
                distance = ''
                pass
                
            try:
                gadgets = flat_soup.find(class_= 'mdi mdi-folder mdi-36px noprint').parent.text.replace('\n', '').replace(' ', '')
            except:
                gadgets = ''
                pass

            try:
                roommates = flat_soup.find(class_= 'col-sm-6').text.replace('\n', '').replace('   ', '')

            except:
                roommates= ''
                pass
                
            try:
                room_description = flat_soup.find(id= 'freitext_0_content').text.replace('\n', '').replace('   ', '')
            except:
                room_description= np.NaN
                pass
            try:
                location_description = flat_soup.find(id= 'freitext_1_content').text.replace('\n', '').replace('   ', '')
            except:
                location_description = np.NaN
                pass

            try:
                flat_live = flat_soup.find(id= 'freitext_2_content').text.replace('\n', '').replace('  ', '')
            except:
                flat_live= np.NaN
                pass
            
            try:
                additional = flat_soup.find(id= 'freitext_3_content').text.replace('\n', '').replace('   ', '')
            except:
                additional= np.NaN
                pass

            try:
                free_from = flat_soup.find(class_= 'col-sm-3').text.replace('\n', '').replace('   ', '')
            except:
                free_from = ''
                pass

            try:
                address = flat_soup.find(class_= 'col-sm-4 mb10').text.replace('\n\n', '').replace('   ', '').replace('Umzugsfirma beauftragen1', '').replace('Adresse', '')
            except:
                address = ''
                pass
            
            #Append the modified scraped text blocks to list

            #Barrierefrei
            list_accessible.append(accessible)
            #Kaltmiete
            list_rent_cold.append(prepare_rent_cold(rent_cold_additional_cost))
            #Nebenkosten
            list_additional_cost.append(prepare_additional_cost(rent_cold_additional_cost))
            #Distanze zu Öffis
            list_distance.append(prepare_distance(distance))
            #Ausstattung der Wohnung
            list_gadgets.append(gadgets)
            #Weibliche Mitbewohnerinnen
            list_female_roommates.append(prepare_female_roommates(roommates))
            #männliche Mitbewohner
            list_male_roommates.append(prepare_male_roommates(roommates))
            #diverse Mitbewohnende
            list_divers_roommates.append(prepare_divers_roommates(roommates))
            #Freitext Zimmerbeschreibung
            list_room_description.append(room_description)
            #Freitext Lagebeschreibung
            list_location_description.append(location_description)
            #Freitext WgLeben
            list_flat_live.append(flat_live)
            #Freitext Generellen Infos
            list_additional.append(additional)
            #RaucherWg oder nicht (Alle sind raucher außer explizite Nichtraucher WGs)
            list_smoker.append(prepare_smoker(roommates))
            #Art des Zusammenlebens
            list_kind_of_flat.append(prepare_kind_of_flat(roommates))
            #Add flat url (unique key)
            list_url.append(flat_url)
            #Einzugsdatum
            list_free_from.append(get_date_from_text(free_from))
            #Postalcode and City name
            list_plz_city.append(get_plz_city(address))
            #Streetaddress
            list_street.append(get_street(address))
    #Add everything to Dataframe at once
    d = {'Street': list_street,'Address': list_plz_city,'rent_cold':list_rent_cold,'additional_cost':list_additional_cost,'free_from': list_free_from, 'distance': list_distance, 'gadgets': list_gadgets,'female_roommates':list_female_roommates,'male_roommates': list_male_roommates,'divers_roommates':list_divers_roommates,'room_description':list_room_description,'location_description': list_location_description,'flat_live':list_flat_live,'additional':list_additional,'smoker':list_smoker,'accessible': list_accessible,'kind_of_flat':list_kind_of_flat,'Url':list_url}
    df = pd.DataFrame(d)
    return df

In [ ]:
#returns the city and the postalcode as one string
def get_plz_city(text):

    pattern = r'\d{5}.*?(?=\n)'

    matches = re.findall(pattern, text, re.DOTALL)
    if(matches):
        for match in matches:
            
            return match
    else:
        return np.NaN

In [ ]:
def get_street(text):
    #Seach for everything between the first two \n
    pattern = r'(?<=\n).*?(?=\n)'

    match = re.search(pattern, text, re.DOTALL)
    if match:
        return (match.group(0))
    else:
        return np.NaN

In [ ]:
def get_date_from_text(text):
    if not text:
        return np.NaN

    pattern = r"\b(\d{2}\.\d{2}\.\d{4})\b"

    matches = re.findall(pattern, text)

    
    match = re.search(pattern, text)
    if match:
        matched_date = match.group(1)
        return matched_date   
    else: 
        return np.NaN

In [ ]:
def prepare_distance(text):
    if not text:
        return np.NaN
    return text[0]

In [ ]:
def prepare_rent_cold(text):
    if not text:
        return np.NaN
    pattern = r":(.*?)€"
    match = re.search(pattern, text)

    if match:
        captured_text = match.group(1)
        return captured_text
    else:
        return np.NaN 

In [ ]:
def prepare_additional_cost(text):
    if not text:
        return 0
    pattern = r"\.\s\s(.*?)€"

    match = re.search(pattern, text)
    if match :
        captured_text = match.group(1)
        if(len(captured_text) <= 4):
            try:
                if captured_text:
                    return(captured_text)
            except:
                return 0  
        else:
            return 0 
    else:
        return 0

In [ ]:
def prepare_divers_roommates(text):
    
    if not text:
        return np.NaN
    
    pattern_divers = r"(\d+)\s+Divers"
    
    #singular & plural
    match_divers = re.search(pattern_divers, text)
    
    if match_divers:
        captured_number = match_divers.group(1)
        return captured_number
    else:
        return 0

In [ ]:
def prepare_female_roommates(text):
    if not text:
        return np.NaN
    #text = "The number 3 Frau is mentioned here."
    pattern_woman = r"(\d+)\s+Frau"
    pattern_women = r"(\d+)\s+Frauen"
    
    
    #singular
    match_woman = re.search(pattern_woman, text)
    #plural
    match_women = re.search(pattern_women, text)

    if match_woman:
        captured_number = match_woman.group(1)
        return captured_number
    elif match_women:
        captured_number = match_women.group(1)
        return captured_number
    else:
        return 0

In [ ]:
def prepare_male_roommates(text):
    if not text:
        return np.NaN
    
    #text = "The number 3 Frau is mentioned here."
    
    pattern_man = r"(\d+)\s+Mann"
    pattern_men = r"(\d+)\s+Männer"
    
    #singular
    match_man = re.search(pattern_man, text)
    #plural
    match_men = re.search(pattern_men, text)

    if match_man:
        captured_number = match_man.group(1)
        return captured_number
    elif match_men:
        captured_number = match_men.group(1)
        return captured_number
    else:
        return 0

In [ ]:
def prepare_smoker(text):
    if not text:
        return np.NaN
    
    pattern1 = r"Rauchen nicht erwünscht"
    pattern2 = r"Rauchen auf dem Balkon erlaubt"
    pattern3 = r"Rauchen im Zimmer erlaubt"
    pattern4 = r"Rauchen überall erlaubt"

    matches1 = re.search(pattern1, text)
    matches2 = re.search(pattern2, text)
    matches3 = re.search(pattern3, text)
    matches4 = re.search(pattern4, text)
    
    if matches1:
        return int(0)
    elif matches2:
        return int(1)
    elif matches3:
        return int(1)
    elif matches4:
        return int(1)
    else:
        return np.NaN

In [ ]:
def prepare_kind_of_flat(text):
    if not text:
        return np.NaN
    
    options = ["gemischte WG", "Studenten-WG", "WG-Neugründung", "Verbindung", "Alleinerziehende", "Senioren-WG",
            "Wohnen für Hilfe", "funktionale WG", "inklusive WG", "Plus-WG", "Azubi-WG", "Wohnheim",
            "Männer-WG", "Mehrgenerationen", "Vegetarisch/Vegan", "Business-WG", "keine Zweck-WG", "WG mit Kindern",
            "LGBTQIA+", "Internationals welcome", "Frauen-WG", "Zweck-WG"]

    # Create the regular expression pattern by joining the phrases with the "|" (OR) operator
    pattern = r"\b(" + "|".join(map(re.escape, options)) + r")\b"

    # Find all occurrences of the phrases in the text
    matches = re.findall(pattern, text)
    matches_as_string= str()
    """
    for i in matches:
        matches_as_string= matches_as_string+str(matches)
    # return the matches
    """
    return(str(matches).replace("'", "").replace("[", "").replace("]", ""))

In [ ]:
def get_price(text):
    match = re.search(r'(.*?)€', text)
    if match:
        return match.group(1)
        
    else:
        return np.NaN()

In [ ]:
#retuns the flat/room size
def get_size(text):
    match = re.search(r'€(.*)', text)
    if match:
        return match.group(1)
        
    else:
        return np.NaN()

In [ ]:
#delets the datevalue out of the text
def replace_date_in_string(text):
    return re.sub(r'\d{2}\.\d{2}\.\d{4}', '', text)

In [ ]:
def get_front_page_data(soup):
    flats = soup.find_all(
                class_="col-sm-8 card_body"
            )
    #create empty lists
    art=[]
    district=[]
    size=[]
    price=[]
    df = pd.DataFrame()
    for idx, x in enumerate(flats):
        #some flats are offert by companies and if you click on the flat you get a list insteat of the normal view
        if 'Verifiziertes Unternehmen'  in str(x.find(class_= 'row noprint bottom').text.replace('\n','').replace('   ','')):
            continue
        else:
            art.append(get_type(str(x.find(class_= 'col-xs-11').text.replace('\n','').replace('   ',''))))
            district.append(get_district(str(x.find(class_= 'col-xs-11').text.replace('\n','').replace(' ','').replace('Leipzig',''))))
            text =replace_date_in_string(x.find(class_= 'row noprint middle').text.replace('\n','').replace('-','').replace(' m²','').replace('   ',''))
            size.append(get_size(text))
            price.append(get_price(text))

    df['type']=art
    df['district']= district  
    df['size']=size
    df['price']=price
    
    return df 

In [ ]:
#returns the type of a flat/room( 1-zimmer wohung, 3er-Wg...)
def get_type(text):
    match = re.search(r'^(.*?)\|', text)
    if match:
        result = match.group(1)
        return result
    else:
        return np.NaN()

In [ ]:
#returns the district of the flat
def get_district(text):
    match = re.search(r'\|([^|]*)\|', text)
    if match:
        return match.group(1)   
    else:
        return np.NaN()

In [ ]:
#navigates the soup of the frontpage to the soup of every flat
def get_subpages_from_soup(soup):
    
    subpage_list=[]
   
    flats = soup.find_all(
            class_="truncate_title noprint"
        )
    
    for idx,flat in enumerate (flats):
            
            #time.sleep(randint(1*100, sleep_sec*100)/100)
            #navigate to subpage and create subpage_Url
            temp = str('https://www.wg-gesucht.de')+str(flat.find('a')['href'])
            if 'asset_id' not in temp:
                subpage_list.append(temp)
            else:
                 continue
   
    return subpage_list